In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "turbidity_df.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("turbidity_df.csv"), header=True, inferSchema=True)

# Show DataFrame
df.show()

+---+---------------+----------------+--------------+------------+-----------+-------------+----------+------------+
|_c0|  DateTimeStamp|LKSPOMET_TotPrcp|LKSPOMET_ATemp|LKSBAWQ_Temp|LKSBAWQ_Sal|LKSBAWQ_Depth|LKSBAWQ_pH|LKSBAWQ_Turb|
+---+---------------+----------------+--------------+------------+-----------+-------------+----------+------------+
|  0|01/01/2018 0:00|             0.0|         -24.5|        null|       null|         null|      null|        null|
|  1|01/01/2018 0:15|             0.0|         -24.7|        null|       null|         null|      null|        null|
|  2|01/01/2018 0:30|             0.0|         -24.8|        null|       null|         null|      null|        null|
|  3|01/01/2018 0:45|             0.0|         -25.0|        null|       null|         null|      null|        null|
|  4|01/01/2018 1:00|             0.0|         -25.2|        null|       null|         null|      null|        null|
|  5|01/01/2018 1:15|             0.0|         -25.2|        nul

In [10]:
server_times = sc.parallelize([('DateTimeStamp').toDF(['ServerTime'])

SyntaxError: incomplete input (3473735387.py, line 1)

In [9]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, date_format

df.select(unix_timestamp(df.ServerTime, 'm/d/yyyy h:mm').alias('ut'))\
  .select(from_unixtime('ut').alias('dty'))\
  .select(date_format('dty', 'M/d/yyyy').alias('Date'),
          date_format('dty', 'h:m:s a').alias('Time'))\
  .show()

AttributeError: 'DataFrame' object has no attribute 'ServerTime'

In [ ]:
# Create our temporary view
df.createOrReplaceTempView('movies')

In [ ]:
# We can perform most any SQL action at this point
# here we are converting the date to a more workable date object
#NOTE: since we are not assigning this to a dataframe the change is not saved.
spark.sql("""SELECT show_id, 
   type, 
   title, 
   country, 
   TO_DATE(date_added, 'MMMM d, yyyy') 
   AS date_added, 
   release_year, 
   rating, 
   duration 
   FROM movies 
   WHERE date_added IS NOT null AND type='Movie'""").show(10)

In [ ]:
# All of the SQL you learned in Unit 6 is available to you in Spark SQL
# Here we are listing out the counts by rating
# NOTE: it is almost NEVER a good idea to "order by" when using Spark with large datasets (more on this in 8.2)
spark.sql("""
  SELECT
    rating,
    count(*) AS number_of_ratings
  FROM movies
  GROUP BY rating
  ORDER BY 2 DESC
  """).show()

In [ ]:
# Let's output a file with just listing for children
# first we will use our spark sql to write to a dataframe

out_df= spark.sql("""
  SELECT 
  title,
  rating,
  date_added,
  duration
  FROM Movies
  WHERE rating IN ('G','PG', 'PG-13')""")

# Make sure we got what we wanted
out_df.show()

In [ ]:
#  As Spark stores the data in partitions, it will also write data in partitions.
#  These partitions will always be stored in a folder with the same name as the file, and that folder may often contain many subfolders or files.
#  Within the partition folder, there will be a file or files that starts with `part-`, these are CSV files. 
# However, they are often not optimal for friendly reading, but can be downloaded to your computer.

out_df.write.csv('movies_out_spark.csv')

In [ ]:
# The easiest work around of the part file output is to take the data to Pandas and write out a CSV.
# This forces the data to the master node and is not recommended unless you have filtered and/or aggregated your data to a reasonable size.

out_df.toPandas().to_csv('movies_out_pandas.csv')